In [1]:
import numpy as np
import rasterio
from sklearn.preprocessing import MinMaxScaler


In [2]:
def read_raster(file_path):
    """Read a raster file and return the data and metadata."""
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the first band
        metadata = src.meta
    return data, metadata

def normalize_raster(data):
    """Normalize raster data to the range [0, 1]."""
    scaler = MinMaxScaler()
    data_flat = data.flatten().reshape(-1, 1)
    normalized_flat = scaler.fit_transform(data_flat)
    return normalized_flat.reshape(data.shape)

def ahp_weights(criteria_weights):
    """Calculate normalized AHP weights from criteria weights."""
    total = sum(criteria_weights)
    return [weight / total for weight in criteria_weights]

def topsis(data, weights):
    """Perform TOPSIS analysis."""
    norm_data = data / np.sqrt((data ** 2).sum(axis=0))
    weighted_data = norm_data * weights

    ideal_best = np.max(weighted_data, axis=0)
    ideal_worst = np.min(weighted_data, axis=0)

    distance_best = np.sqrt(((weighted_data - ideal_best) ** 2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_data - ideal_worst) ** 2).sum(axis=1))

    scores = distance_worst / (distance_best + distance_worst)
    return scores

def weighted_overlay(rasters, weights):
    """Perform weighted overlay of rasters."""
    combined = np.zeros_like(rasters[0])
    for i, raster in enumerate(rasters):
        combined += raster * weights[i]
    return combined

def average_weighted(rasters):
    """Perform average weighted overlay of rasters."""
    return np.mean(rasters, axis=0)

def save_raster(output_path, data, metadata):
    """Save data to a new raster file."""
    metadata.update({"dtype": "float32"})
    with rasterio.open(output_path, "w", **metadata) as dst:
        dst.write(data, 1)


In [4]:
NDVI = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\NDVI.tif"
EVI  = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\NDWI.tif"
NDWI = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\EVI.tif"
NIR = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\NIR.tif"
RED = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\GREEN.tif"
GREEN = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\RED.tif"

In [5]:
# File paths to input TIFFs
file_paths = [NDVI, EVI, NDWI, NIR, GREEN, RED]



In [6]:
# Read and normalize raster data
rasters = []
metadata = None
for file_path in file_paths:
    data, metadata = read_raster(file_path)
    normalized_data = normalize_raster(data)
    rasters.append(normalized_data)

# Define criteria weights (example values, sum must equal 1)
criteria_weights = [0.2, 0.3, 0.1, 0.25, 0.15, 0.3]

# Normalize AHP weights
weights = ahp_weights(criteria_weights)

# Stack rasters into a single array for MCDM/TOPSIS analysis
stacked_rasters = np.stack(rasters, axis=2)
rows, cols, num_criteria = stacked_rasters.shape
stacked_flat = stacked_rasters.reshape(rows * cols, num_criteria)

# Perform TOPSIS analysis
scores_topsis = topsis(stacked_flat, weights)
suitability_map_topsis = scores_topsis.reshape(rows, cols)

# Perform Weighted Overlay analysis
weighted_map = weighted_overlay(rasters, weights)

# Perform Average Weighted analysis
average_weighted_map = average_weighted(rasters)



In [7]:
# Save output rasters
output_path_topsis = "D:/Acres/December 2024/Assignment 3/Results/habitat_suitability_topsis.tif"
output_path_weighted = "D:/Acres/December 2024/Assignment 3/Results/habitat_suitability_weighted_overlay.tif"
output_path_avg_weighted = "D:/Acres/December 2024/Assignment 3/Results/habitat_suitability_avg_weighted.tif"

save_raster(output_path_topsis, suitability_map_topsis, metadata)
save_raster(output_path_weighted, weighted_map, metadata)
save_raster(output_path_avg_weighted, average_weighted_map, metadata)

print(f"TOPSIS-based habitat suitability map saved to {output_path_topsis}")
print(f"Weighted overlay habitat suitability map saved to {output_path_weighted}")
print(f"Average weighted habitat suitability map saved to {output_path_avg_weighted}")

TOPSIS-based habitat suitability map saved to D:/Acres/December 2024/Assignment 3/Results/habitat_suitability_topsis.tif
Weighted overlay habitat suitability map saved to D:/Acres/December 2024/Assignment 3/Results/habitat_suitability_weighted_overlay.tif
Average weighted habitat suitability map saved to D:/Acres/December 2024/Assignment 3/Results/habitat_suitability_avg_weighted.tif


In [14]:
#AHP

In [13]:
from functools import reduce
import numpy as np
import rasterio

# Define the AHP method function
def ahp_method(dataset, wd='m'):
    inc_rat = np.array([0, 0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51, 1.48, 1.56, 1.57, 1.59])
    X = np.copy(dataset)
    weights = np.zeros(X.shape[1])

    if wd == 'm' or wd == 'mean':
        weights = np.mean(X / np.sum(X, axis=0), axis=1)
        vector = np.sum(X * weights, axis=1) / weights
        lamb_max = np.mean(vector)
    elif wd == 'g' or wd == 'geometric':
        for i in range(X.shape[1]):
            weights[i] = reduce((lambda x, y: x * y), X[i, :])**(1 / X.shape[1])
        weights = weights / np.sum(weights)
        vector = np.sum(X * weights, axis=1) / weights
        lamb_max = np.mean(vector)
    elif wd == 'me' or wd == 'max_eigen':
        eigenvalues, eigenvectors = np.linalg.eig(X)
        eigenvalues_real = np.real(eigenvalues)
        lamb_max_index = np.argmax(eigenvalues_real)
        lamb_max = eigenvalues_real[lamb_max_index]
        principal_eigenvector = np.real(eigenvectors[:, lamb_max_index])
        weights = principal_eigenvector / principal_eigenvector.sum()

    cons_ind = (lamb_max - X.shape[1]) / (X.shape[1] - 1)
    rc = cons_ind / inc_rat[X.shape[1]]
    return weights, rc

# Define the pairwise comparison matrix
criteria_matrix = np.array([
    [1, 3, 0.5, 2, 1, 4],
    [0.33, 1, 0.25, 0.5, 0.33, 2],
    [2, 4, 1, 3, 2, 5],
    [0.5, 2, 0.33, 1, 0.5, 3],
    [1, 3, 0.5, 2, 1, 4],
    [0.25, 0.5, 0.2, 0.33, 0.25, 1]
])

# Calculate AHP weights
ahp_weights, consistency_ratio = ahp_method(criteria_matrix, wd='m')

if consistency_ratio > 0.1:
    print("Warning: Consistency ratio is too high. Revise the pairwise comparison matrix.")
else:
    print(f"AHP weights: {ahp_weights}")
    print(f"Consistency ratio: {consistency_ratio}")

# Raster file paths
NDVI = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\NDVI.tif"
EVI = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\EVI.tif"
NDWI = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\NDWI.tif"
NIR = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\NIR.tif"
RED = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\RED.tif"
GREEN = r"D:\Acres\December 2024\Assignment 3\Reclassification Data\GREEN.tif"

# Reading rasters into arrays
def read_raster(file_path):
    with rasterio.open(file_path) as src:
        return src.read(1), src.transform, src.crs

# Read each raster file
ndvi, ndvi_transform, ndvi_crs = read_raster(NDVI)
evi, _, _ = read_raster(EVI)
ndwi, _, _ = read_raster(NDWI)
nir, _, _ = read_raster(NIR)
red, _, _ = read_raster(RED)
green, _, _ = read_raster(GREEN)

# Combine rasters into a single stack
stack = np.array([ndvi, evi, ndwi, nir, red, green])

# Normalize each layer to a 0-1 range
def normalize(layer):
    return (layer - np.min(layer)) / (np.max(layer) - np.min(layer))

stack_normalized = np.array([normalize(layer) for layer in stack])

# Apply AHP weights to normalized stack
weighted_stack = np.zeros_like(stack_normalized[0])
for i, weight in enumerate(ahp_weights):
    weighted_stack += stack_normalized[i] * weight

# Save the weighted result to a new raster file
output_path = r"D:\Acres\December 2024\Assignment 3\Results\Weighted_Suitability.tif"

with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=weighted_stack.shape[0],
    width=weighted_stack.shape[1],
    count=1,
    dtype=weighted_stack.dtype,
    crs=ndvi_crs,
    transform=ndvi_transform
) as dst:
    dst.write(weighted_stack, 1)

print(f"Weighted suitability map saved to: {output_path}")


AHP weights: [0.20546767 0.07596862 0.34105314 0.12247477 0.20546767 0.04956814]
Consistency ratio: 0.010685467139109
Weighted suitability map saved to: D:\Acres\December 2024\Assignment 3\Results\Weighted_Suitability.tif
